### Objetivo

Esse notebook tem como objetivo realizar uma análise de sentimentos do dataset da [amazon](http://deepyeti.ucsd.edu/jianmo/amazon/index.html). Essa vai ser uma série de 10 projetos de NLP onde esse é o primeiro projeto. 

Análise de sentimento é um problema difícil, mas tratando-se de NLP é um dos problemas mais simples de serem resolvidos.
Neste projeto, pretendo:

    1 - Analisar os dados.
    2 - Aplicar um modelo usando rede neural recorrente
    3 - Aplicar um modelo usando LSTM
    

In [26]:
import pandas as pd
import tensorflow as tf
import nltk
import re
from tqdm import tqdm
from IPython.display import clear_output
import numpy as np
import sklearn

chunksize = 10000
data_size = 50000
reader = pd.read_json("./dataset/Video_Games.json", chunksize=chunksize, lines=True)

### Dados a serem usados

Bem, essa base de dados tem 1.6 GB de dados. Não tenho como guardar tudo isso em memória, então decidi só carregar Nk amostras. A seguir, eu vou carregar mais e garantir que existam pelo menos Nk frases de cada nota. Esse numero de 5Nk amostras é um número que acredito que será possível ter um experimento interessante.

In [28]:
data = []
for _ in reader:
    data.append(_)
    break
data = data[0]
print(data.head())
data.columns

       overall  verified   reviewTime      reviewerID        asin  \
10000        1      True   12 5, 2014   A5Z8JVR5415AR  B00000JDFT   
10001        1      True  10 19, 2014   AMNHAB7BQ3BH9  B00000JDFT   
10002        5      True  08 26, 2014  A140M20XMQANKX  B00000JDFT   
10003        5      True   07 8, 2014   AIH4T1700UJ35  B00000JDFT   
10004        5      True  12 17, 2013  A1YE2KK40M2TTG  B00000JDFT   

               reviewerName  \
10000                 chris   
10001  Albert L. Horney Jr.   
10002          Michael Ault   
10003        Chris Brunelle   
10004                 Wilco   

                                              reviewText  \
10000  Defective product---... Microsoft made these s...   
10001     Could never get it to work on any of my games.   
10002              well its a sidewinder what can ya say   
10003                                              great   
10004  Like many others, I consider this joystick to ...   

                                  sum

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')

#### Percebemos que as notas não estão balanceadas.

Isso pode acabar trazendo uma tendência nas avaliações. Vou tentar equilibrar os dados.

In [231]:
data["overall"].value_counts()

5    6678
4    1443
1     915
3     636
2     328
Name: overall, dtype: int64

In [232]:
samples_max_size = chunksize

#### Garantindo que existam samples_max_size amostras para cada nota.

In [3]:
def equalize_samples(reader, data, samples_max_size, data_size):
    for _ in reader:
        validate = data["overall"].value_counts().sum()
        if validate == data_size:
            break
        for i in range(1,6):
            aux = _.groupby("overall").filter(lambda x: pd.Series([i]).isin(x["overall"]).all())[data.columns]
            curr_class_size = data["overall"].value_counts()[i]
            if curr_class_size + aux.shape[0] < samples_max_size:
                #adiciona
                data = pd.concat([data, aux], axis = 0)
            elif curr_class_size < samples_max_size:
                #adiciona parcial
                offset = curr_class_size + aux.shape[0] - samples_max_size
                data = pd.concat([data, aux[offset:]], axis = 0)
            else:
                clear_output(wait=True)
                print(data["overall"].value_counts())
                continue
            clear_output(wait=True)
            print(data["overall"].value_counts())
    return data

In [235]:
data.head(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1,True,"06 9, 2014",A21ROB4YDOZA5P,0439381673,Mary M. Clark,I used to play this game years ago and loved i...,Did not like this,1402272000,NaN,NaN,NaN
1,3,True,"05 10, 2014",A3TNZ2Q5E7HTHD,0439381673,Sarabatya,The game itself worked great but the story lin...,Almost Perfect,1399680000,NaN,NaN,NaN
2,4,True,"02 7, 2014",A1OKRM3QFEATQO,0439381673,Amazon Customer,I had to learn the hard way after ordering thi...,DOES NOT WORK WITH MAC OS unless it is 10.3 or...,1391731200,15.0,NaN,NaN


#### Lembre-se de remover os NaNs dos dados que serão usados.

In [236]:
data.isna().sum()

overall               0
verified              0
reviewTime            0
reviewerID            0
asin                  0
reviewerName          2
reviewText            6
summary               7
unixReviewTime        0
vote              32765
style             33126
image             49739
dtype: int64

In [237]:
data.dropna(subset=["reviewText"],  axis=0, inplace=True)
data.isna().sum()

overall               0
verified              0
reviewTime            0
reviewerID            0
asin                  0
reviewerName          2
reviewText            0
summary               7
unixReviewTime        0
vote              32763
style             33122
image             49739
dtype: int64

#### Alguns elementos foram deletados, mas ainda temos uma distribuição legal entre as notas.

In [238]:
data["overall"].value_counts()

4    10000
5    10000
2    10000
3     9999
1     9995
Name: overall, dtype: int64

#### Processando a sentença

Aqui foi criado esse método para gerar a sentença sem as palavras que são irrelevantes para análise de sentimento (stopwords) e também foi usada uma técnica de stem para preservar somente o radical aproximado das palavras.

In [4]:
def process_sentence(sentence, padding=30):
    stopwords = nltk.corpus.stopwords
    stemer = nltk.stem.PorterStemmer()
    processed = nltk.word_tokenize(sentence[:padding])
    processed = [stemer.stem(word) for word in processed if word not in stopwords.words("english")] + padding * ["<PAD>"]
    return processed[:padding]

print(process_sentence(data["reviewText"][100], padding=50))

['game', 'amaz', 'period', '.', 'i', 'creat', 'review', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


#### Tokenizando a sentença processada

Aqui é gerado os tokens de um corpus. Isso é, aqui é construído o vocabulário. Com os dicionários resultantes é possível construir um método para tokenizar e detokenizar.

Como as palavras de pausa foram removidas, a detokenização será incompleta. Isto significa que não será possível reconstruir a frase toda. 

In [7]:
def make_vocabulary(corpus, padding=30, max_vocab_size=2000):
    vocabulary = {'<PAD>':0, '<UNK>':1}
    rvocabulary = {0:'<PAD>', 1:'<UNK>'}
    fvocabulary = {'<PAD>':0, '<UNK>':0}
    index = 2
    for sentence in tqdm(corpus):
        processed = process_sentence(sentence, padding=padding)
        for word in processed:
            if word not in vocabulary.keys():
                vocabulary[word] = index
                rvocabulary[index] = word
                fvocabulary[word] = 1
                index += 1
            else:
                fvocabulary[word] += 1
    fvocabulary = dict(sorted(fvocabulary.items(), key=lambda item: item[1], reverse=True))
    words_by_freq = list(fvocabulary.keys())[:max_vocab_size]
    index = 2
    aux = {'<PAD>':0, '<UNK>':1}
    for word in vocabulary.keys():
        if word in words_by_freq and word not in ['<PAD>', '<UNK>']:
            aux[word] = index
            index+=1
    vocabulary = aux
    return rvocabulary, vocabulary, fvocabulary

vocab = make_vocabulary([data["reviewText"][100]], padding=50, max_vocab_size=4)
print(vocab[0])
print(vocab[1])
print(vocab[2])

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 328.68it/s]

{0: '<PAD>', 1: '<UNK>', 2: 'game', 3: 'amaz', 4: 'period', 5: '.', 6: 'i', 7: 'creat', 8: 'review'}
{'<PAD>': 0, '<UNK>': 1, 'game': 2, 'amaz': 3, 'period': 4}
{'<PAD>': 43, 'game': 1, 'amaz': 1, 'period': 1, '.': 1, 'i': 1, 'creat': 1, 'review': 1, '<UNK>': 0}


In [6]:
def tokenize(sentence, vocabulary, padding = 30):
    sentence = process_sentence(sentence, padding=padding)
    return [vocabulary[word] if word in vocabulary else vocabulary["<UNK>"] for word in sentence]
def detokenize(sentence, rvocabulary, padding=30):
    return [rvocabulary[token] for token in sentence]
tokenized = tokenize(data["reviewText"][100], vocab[1], padding=50)
print(tokenized)
detokenized = detokenize(tokenized, vocab[0], padding=50)
print(detokenized)

[2, 3, 4, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['game', 'amaz', 'period', '<UNK>', '<UNK>', '<UNK>', '<UNK>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [299]:
rvocabulary, vocabulary, fvocabulary = make_vocabulary(data["reviewText"].to_numpy(), padding=50, max_vocab_size=5000)

100%|███████████████████████████████████████████████████████████████████████████| 49994/49994 [02:03<00:00, 405.28it/s]


In [300]:
print(list(vocabulary.items())[:20])

[('<PAD>', 0), ('<UNK>', 1), ('i', 2), ('use', 3), ('play', 4), ('game', 5), ('year', 6), ('ago', 7), ('love', 8), ('.', 9), ('the', 10), ('work', 11), ('great', 12), ('stori', 13), ('line', 14), ('vi', 15), ('learn', 16), ('hard', 17), ('way', 18), ('order', 19)]


In [301]:
tokenized = tokenize(data["reviewText"][100], vocabulary, padding=50)
print(tokenized)
detokenized = detokenize(tokenized, rvocabulary, padding=50)
print(detokenized)

[5, 186, 258, 9, 2, 259, 260, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['game', 'there', 'dirt2', '.', 'i', 'pc', 'cross', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


#### Agora que o vocabulário foi criado

Agora que o vocabulário foi criado, podemos preparar os dados para treinar o modelo. E por preparar os dados eu me refiro a fazer a divisão dos dados em treino e teste. 

In [321]:
x = data["reviewText"].to_numpy()
x = np.array([tokenize(text, vocabulary, padding=50) for text in tqdm(x)])

100%|███████████████████████████████████████████████████████████████████████████| 49994/49994 [02:04<00:00, 401.99it/s]


In [373]:
y = data["overall"].to_numpy() - 1 #Corrigindo intervalo para [0,5)

In [384]:
xtrain, xtest, ytrain, ytest = sklearn.model_selection.train_test_split(x, y, train_size=.9)

In [385]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((44994, 50), (5000, 50), (44994,), (5000,))

#### Agora os dados estão 100%

Agora os dados estão 100% preparados para entrar em qualquer modelo. A etapa a seguir vai ser a de construir o modelo que vamos usar aqui. 

Observação:
No modelo abaixo usei logsoftmax ao invés de softmax. Isso ajuda a previnir operar em cima de produtos de valores muito pequenos. No caso, trabalhando com logit teremos somatório de valores, que ajuda a estabilizar o aprendizado.

In [10]:
class LogSoftmax(tf.keras.layers.Softmax):
    def __init__(self):
        super(LogSoftmax, self).__init__()
        
    def call(self, inputs):
        return tf.math.log(super(LogSoftmax, self).call(inputs))

In [399]:
vocab_size=5000
embedding_dim=128
rnn = tf.keras.models.Sequential(
    [
        #Lembrando que são 5000 tokens + 2 que é o <PAD> e <UNK>
        tf.keras.layers.Embedding(vocab_size + 2, embedding_dim, input_length=x.shape[1]),
        tf.keras.layers.SimpleRNN(128),
        tf.keras.layers.Dropout(.4),
        tf.keras.layers.Dense(5),
        LogSoftmax()
    ]
)
rnn.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 50, 128)           640256    
_________________________________________________________________
simple_rnn_20 (SimpleRNN)    (None, 128)               32896     
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 645       
_________________________________________________________________
log_softmax_3 (LogSoftmax)   (None, 5)                 0         
Total params: 673,797
Trainable params: 673,797
Non-trainable params: 0
_________________________________________________________________


In [400]:
rnn.compile(optimizer='adam', 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [401]:
history = rnn.fit(
    x = xtrain,
    y = ytrain, 
    epochs=20, 
    batch_size=256,
    validation_data=(xtest,ytest)
)

Epoch 1/20
176/176 [==============================] - 23s 126ms/step - loss: 1.5205 - accuracy: 0.3098 - val_loss: 1.4332 - val_accuracy: 0.3658
Epoch 2/20
176/176 [==============================] - 23s 130ms/step - loss: 1.3786 - accuracy: 0.4051 - val_loss: 1.4216 - val_accuracy: 0.3772
Epoch 3/20
176/176 [==============================] - 23s 130ms/step - loss: 1.3005 - accuracy: 0.4525 - val_loss: 1.4094 - val_accuracy: 0.3888
Epoch 4/20
176/176 [==============================] - 23s 130ms/step - loss: 1.2097 - accuracy: 0.5066 - val_loss: 1.4658 - val_accuracy: 0.3700
Epoch 5/20
176/176 [==============================] - 22s 128ms/step - loss: 1.1343 - accuracy: 0.5493 - val_loss: 1.4960 - val_accuracy: 0.3734
Epoch 6/20
176/176 [==============================] - 22s 128ms/step - loss: 1.0719 - accuracy: 0.5806 - val_loss: 1.5528 - val_accuracy: 0.3636
Epoch 7/20
176/176 [==============================] - 23s 129ms/step - loss: 1.0265 - accuracy: 0.6031 - val_loss: 1.5794 - val_ac

#### A performance da RNN 

A performance da RNN foi bem ruim. Tendeu total ao overfitting. Isso quem sabe pode ser resolvido com mais dados. Também o padding que foi escolhido é muito pequeno.

Eu ia até testar a LSTM, mas não faz muito sentido, afinal não estou usando sequências longas. Então vamos reconstruír os dados e usar o mesmo modelo. 

Também vou aproveitar para modificar o tamanho do vocabulário

In [402]:
vocab_size=8000
padding=150
rvocabulary, vocabulary, fvocabulary = make_vocabulary(data["reviewText"].to_numpy(), paddingsamples_max_sizeng, max_vocab_size=vocab_size)
x = data["reviewText"].to_numpy()
x = np.array([tokenize(text, vocabulary, padding=padding) for text in tqdm(x)])
y = data["overall"].to_numpy() - 1 #Corrigindo intervalo para [0,5)
xtrain, xtest, ytrain, ytest = sklearn.model_selection.train_test_split(x, y, train_size=.9)

embedding_dim=128
rnn = tf.keras.models.Sequential(
    [
        #Lembrando que são 5000 tokens + 2 que é o <PAD> e <UNK>
        tf.keras.layers.Embedding(vocab_size + 2, embedding_dim, input_length=x.shape[1]),
        tf.keras.layers.SimpleRNN(128),
        tf.keras.layers.Dropout(.4),
        tf.keras.layers.Dense(5),
        LogSoftmax()
    ]
)
print(rnn.summary())

rnn.compile(optimizer='adam', 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = rnn.fit(
    x = xtrain,
    y = ytrain, 
    epochs=20, 
    batch_size=256,
    validation_data=(xtest,ytest)
)

100%|███████████████████████████████████████████████████████████████████████████| 49994/49994 [05:07<00:00, 162.70it/s]


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 150, 128)          1024256   
_________________________________________________________________
simple_rnn_21 (SimpleRNN)    (None, 128)               32896     
_________________________________________________________________
dropout_23 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 5)                 645       
_________________________________________________________________
log_softmax_4 (LogSoftmax)   (None, 5)                 0         
Total params: 1,057,797
Trainable params: 1,057,797
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
176/176 [==============================] - 60s 339ms/step - loss: 1.6158 - accuracy: 0.2027 -

KeyboardInterrupt: 

#### Infelizmente Parei a Execução

Para minha infelicidade ficou tão ruim quanto o LSTM que eu antes havia testado. Ou seja, por mais que eu tenha recebido mais dados, não estou conseguindo generalizar bem com esses dados. 

Aumentar a riquesa do vocabulário se provou ineficiente. E aumentar o tamanho das sentenças também. E se aumentarmos o total de dados ? 

Abaixo eu tento fazer isso. E verifico com vocabulário de 5k e padding 50 se melhorou alguma coisa.

In [403]:
chunksize = 25000
data_size = 50000

reader = pd.read_json("./dataset/Video_Games.json", chunksize=chunksize, lines=True)

data = []
for _ in reader:
    data.append(_)
    break
data = data[0]

samples_max_size = chunksize

for _ in reader:
    validate = data["overall"].value_counts().sum()
    if validate == data_size:
        break
    for i in range(1,6):
        aux = _.groupby("overall").filter(lambda x: pd.Series([i]).isin(x["overall"]).all())[data.columns]
        curr_class_size = data["overall"].value_counts()[i]
        if curr_class_size + aux.shape[0] < samples_max_size:
            #adiciona
            data = pd.concat([data, aux], axis = 0)
        elif curr_class_size < samples_max_size:
            #adiciona parcial
            offset = curr_class_size + aux.shape[0] - samples_max_size
            data = pd.concat([data, aux[offset:]], axis = 0)
        else:
            clear_output(wait=True)
            print(data["overall"].value_counts())
            continue
        clear_output(wait=True)
        print(data["overall"].value_counts())

data.dropna(subset=["reviewText"],  axis=0, inplace=True)

vocab_size=5000
padding=50
rvocabulary, vocabulary, fvocabulary = make_vocabulary(data["reviewText"].to_numpy(), padding=padding, max_vocab_size=vocab_size)
x = data["reviewText"].to_numpy()
x = np.array([tokenize(text, vocabulary, padding=padding) for text in tqdm(x)])
y = data["overall"].to_numpy() - 1 #Corrigindo intervalo para [0,5)
xtrain, xtest, ytrain, ytest = sklearn.model_selection.train_test_split(x, y, train_size=.9)

embedding_dim=128
rnn = tf.keras.models.Sequential(
    [
        #Lembrando que são 5000 tokens + 2 que é o <PAD> e <UNK>
        tf.keras.layers.Embedding(vocab_size + 2, embedding_dim, input_length=x.shape[1]),
        tf.keras.layers.SimpleRNN(128),
        tf.keras.layers.Dropout(.4),
        tf.keras.layers.Dense(5),
        LogSoftmax()
    ]
)
print(rnn.summary())

rnn.compile(optimizer='adam', 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = rnn.fit(
    x = xtrain,
    y = ytrain, 
    epochs=20, 
    batch_size=256,
    validation_data=(xtest,ytest)
)

1    25000
3    25000
4    25000
5    25000
2    25000
Name: overall, dtype: int64


100%|█████████████████████████████████████████████████████████████████████████| 124981/124981 [04:59<00:00, 417.35it/s]


Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 50, 128)           640256    
_________________________________________________________________
simple_rnn_22 (SimpleRNN)    (None, 128)               32896     
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 5)                 645       
_________________________________________________________________
log_softmax_5 (LogSoftmax)   (None, 5)                 0         
Total params: 673,797
Trainable params: 673,797
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
440/440 [==============================] - 53s 118ms/step - loss: 1.4671 - accuracy: 0.3438 - val

#### Pelo visto...

Mesmo aumentando a quantidade de dados não tivemos um resultado evidentemente melhor que o ultimo. A próxima estratégia que planejo é considerar o nome de quem fez o post como parte da mensagem.

#### Um passo pra trás

Como eu não estive tendo bons resultados tomei a decisão de tentar fazer uma classificação binária. Já havia tentado transformar notas altas em boas e baixas em ruim, mas ainda assim ficou muito nebuloso.

Vamos tentar aqui trabalhar agora somente com notas máximas e notas mínimas, e ver o resultado.

In [24]:
chunksize = 5000
data_size = 5*chunksize

reader = pd.read_json("./dataset/Video_Games.json", chunksize=chunksize, lines=True)

data = []
for _ in reader:
    data.append(_)
    break
data = data[0]

samples_max_size = chunksize

data = equalize_samples(reader,data,samples_max_size,data_size)

data = data[data.overall != 2]
data = data[data.overall != 3]
data = data[data.overall != 4]
data.dropna(subset=["reviewText"],  axis=0, inplace=True)
clear_output(wait=True)
print(data["overall"].value_counts())

vocab_size=8000
padding=50
rvocabulary, vocabulary, fvocabulary = make_vocabulary(data["reviewText"].to_numpy(), padding=padding, max_vocab_size=vocab_size)
x = data["reviewText"].to_numpy()
x = np.array([tokenize(text, vocabulary, padding=padding) for text in tqdm(x)])
print(x[:5])
y = (data["overall"].to_numpy() == 5)*1 #Corrigindo intervalo para {0,1}
print(y[:5])
xtrain, xtest, ytrain, ytest = sklearn.model_selection.train_test_split(x, y, train_size=.9)

5    5000
1    4995
Name: overall, dtype: int64


100%|█████████████████████████████████████████████████████████████████████████████| 9995/9995 [00:22<00:00, 442.44it/s]

[[ 2  3  4  5  6  7  8  9  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [10 11 12 13 14  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [15 16 17 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [19 20 21 22 23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 2 24 25 26  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]
[0 0 1 1 1]


In [25]:
embedding_dim=128
rnn = tf.keras.models.Sequential(
    [
        #Lembrando que são 5000 tokens + 2 que é o <PAD> e <UNK>
        tf.keras.layers.Embedding(vocab_size + 2, embedding_dim, input_length=x.shape[1]),
        tf.keras.layers.SimpleRNN(128),
        tf.keras.layers.Dropout(.4),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(2),
        LogSoftmax()
    ]
)
print(rnn.summary())

rnn.compile(optimizer='adam', 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = rnn.fit(
    x = xtrain,
    y = ytrain, 
    epochs=20, 
    batch_size=256,
    validation_data=(xtest,ytest)
)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 50, 128)           1024256   
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 130       
_________________________________________________________________
log_softmax_7 (LogSoftmax)   (None, 2)                

#### Ok, agora tivemos um resultado. 78% de acurácia na validação (5000 vocab_size, 50 padding, 25k samples)

Enquanto eu estava testando, mesmo somente com 2 classes, não conseguia progredir no treino. Só consegui fazer progresso quando eu adicionei uma nova camada ReLu entre a RNN e a definição de classes. 

E realmente, havia um defeito muito grande pois a ReLu força os valores a serem positivos. Sem fazer isso a gente vai acabar tirando log de valores negativos usando a LogSoftmax, causando assim erro.

#### Se eu aumento os parâmetros a rede se torna ineficiente (8000 vocab_size, 150 padding, 5k samples)

Essa ineficiência tem mais haver com o <b>padding</b>, visto que mantendo os outros parâmetros ainda foi possível atingir a mesma performance de <b>78% de acurácia na validação</b>

Sei que isso acontece porque existem muitas revisões que contém um total de texto muito abaixo do padding. Será que uma LSTM seria capaz de resolver esse problema nessa mesma arquitetura ?

#### Antes, vamos testar a LSTM com (8000 vocab_size, 50 padding, 5k samples)

In [28]:
embedding_dim=128
rnn = tf.keras.models.Sequential(
    [
        #Lembrando que são 5000 tokens + 2 que é o <PAD> e <UNK>
        tf.keras.layers.Embedding(vocab_size + 2, embedding_dim, input_length=x.shape[1]),
        tf.keras.layers.GRU(128),
        tf.keras.layers.Dropout(.4),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(2),
        LogSoftmax()
    ]
)
print(rnn.summary())

rnn.compile(optimizer='adam', 
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = rnn.fit(
    x = xtrain,
    y = ytrain, 
    epochs=20, 
    batch_size=256,
    validation_data=(xtest,ytest),
    verbose=False
)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 50, 128)           1024256   
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 130       
_________________________________________________________________
log_softmax_10 (LogSoftmax)  (None, 2)               

#### Parei a execução (Testei com LSTM e GRU)

A LSTM não consegue performar tão bem nos dados com os parâmetros citados acima. Da mesma forma, a GRU não se comportou bem. Eu acredito que a dificuldade encontrada tá associada à rede. Porque com uma rede mais simples foi possível generalizar e conseguir uma acurácia de validação acima da média.

In [33]:
history.history["accuracy"][-1]

0.49849915504455566

#### Vou tentar mecher na rede

Vou ver se consigo generalizar com os mesmos dados, mas modificando a rede. Abaixo vou colocar somente a rede final.

In [24]:
chunksize = 5000
data_size = 5*chunksize

reader = pd.read_json("./dataset/Video_Games.json", chunksize=chunksize, lines=True)

data = []
for _ in reader:
    data.append(_)
    break
data = data[0]

samples_max_size = chunksize

data = equalize_samples(reader,data,samples_max_size,data_size)

data = data[data.overall != 2]
data = data[data.overall != 3]
data = data[data.overall != 4]
data.dropna(subset=["reviewText"],  axis=0, inplace=True)
clear_output(wait=True)
print(data["overall"].value_counts())

vocab_size=8000
padding=30
rvocabulary, vocabulary, fvocabulary = make_vocabulary(data["reviewText"].to_numpy(), padding=padding, max_vocab_size=vocab_size)
x = data["reviewText"].to_numpy()
x = np.array([tokenize(text, vocabulary, padding=padding) for text in tqdm(x)])
print(x[:5])
y = pd.get_dummies((data["overall"].to_numpy() == 5)*1).to_numpy() #Corrigindo intervalo para {0,1}
print(y[:5])
xtrain, xtest, ytrain, ytest = sklearn.model_selection.train_test_split(x, y, train_size=.9)

5    5000
1    4995
Name: overall, dtype: int64


100%|█████████████████████████████████████████████████████████████████████████████| 9995/9995 [00:15<00:00, 650.57it/s]


[[ 2  3  4  5  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 7  8  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [10 11 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [13 14 15 16 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]
 [ 2 18 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0]]
[[1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]]


In [25]:
embedding_dim=128
rnn = tf.keras.models.Sequential(
    [
        #Lembrando que são 5000 tokens + 2 que é o <PAD> e <UNK>
        tf.keras.layers.Embedding(vocab_size + 2, embedding_dim, input_length=x.shape[1]),
        tf.keras.layers.LSTM(200),
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
print(rnn.summary())

rnn.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = rnn.fit(
    x = xtrain,
    y = ytrain, 
    epochs=20, 
    batch_size=64,
    validation_data=(xtest,ytest),
),

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 30, 128)           1024256   
_________________________________________________________________
lstm_12 (LSTM)               (None, 200)               263200    
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 402       
Total params: 1,287,858
Trainable params: 1,287,858
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
141/141 [==============================] - 12s 76ms/step - loss: 0.6218 - accuracy: 0.6110 - val_loss: 0.4659 - val_accuracy: 0.7610
Epoch 2/20
141/141 [==============================] - 10s 70ms/step - loss: 0.4255 - accuracy: 0.8116 - val_loss: 0.4851 - val_accuracy: 0.7660
Epoch 3/20
141/141 [==============================] - 10s 73ms/step - loss: 0.35

#### Aparentemente eu estava vacilando.

Garantir one_hot_encoding na saída dos dados foi fundamental para o LSTM  funcionar. Feito isso, deu bom o LSTM. Agora finalmente da pra finalizar esse notebook. 

Vou passar a limpo para outro notebook com um experimento único. 